In [208]:
# import packages
import csv
import pandas as pd
import re
import datetime
import io
from io import StringIO
import string
import numpy as np
import re

In [209]:
#Please edit pathway before use
import_path="set_import_path"

export_path="set_export+path"
#Dictionary
location ={
    1: 'Augusta University',
    2: 'Emory University',
}

patient ={
    4:'CFBR-AU-0008',
    5:'CFBR-AU-0002',
    6:'CFBR-AU-0004',
    7:'CFBR-AU-0005',
    8:'CFBR-AU-0003',
    9:'CFBR-AU-0006',
    10:'CFBR-AU-0007',
    11:'CFBR-EM-0100'
}

bristol_stool_chart_complete ={
    0: 'Incomplete',
    1: 'Unverified',
    2: 'Complete',
}

extremity_pacqol ={
    0:'Not at all',
    1:'A little bit',
    2:'Moderately',
    3:'Quite a bit',
    4:'Extremely',
}

frequency_pacqol ={
    0:'None of the time',
    1:'A little of the time',
    2:'Some of the time',
    3:'Most of the time',
    4:'All of the time',
}

reverse_score_pacqol ={
    0:4,
    1:3,
    2:2,
    3:1,
    4:0,
}

In [210]:
#PAC-QOL_DiabetesGINutritionA_DATA_2021-09-09_1321
#Read in table
sample = pd.read_csv (import_path + 'PAC-QOL_DiabetesGINutritionA_DATA_2021-09-09_1321.csv')
sample = sample.replace(np.nan, '', regex = True)

#rename columns   
sample.rename(columns={
'record_id':'patient_id',
'redcap_survey_identifier':'Survey Identifier',
'patient_assessment_of_constipation_pacqol_timestamp':'Survey Timestamp',
'survey_date':'survey_date_pac_qol',
'change_routine_pac_qol_2':'change_routine_pac_qol'
}, inplace=True)

In [211]:
# functions for score calculation
def calc_score(input):
    '''Takes a list of strings（column names） as input， 
    returns a list of average scores '''
    ls=[]
    for item in input:
        arr=sample[item].to_numpy()
        int_arr = [int(i) for i in arr]
        ls.append(int_arr)
        
    ls=np.array(ls)
    score=sum(ls,0)/len(input)
    return(score)

def rev_score(col_name):
    '''returns an array of reversed socring of the input column name'''
    arr_reverse=[]
    arr=sample[col_name].to_numpy()
    for item in arr:
        arr_reverse.append(reverse_score_pacqol [item])
        
    arr_reverse=np.array(arr_reverse)
    return arr_reverse

def calc_reverse_scores(input):
    '''Takes a list of strings（column names） as input， 
    returns an array of sum reversed scores ''' 
    ls=[]
    for item in input:
        arr=rev_score(item)
        ls.append(arr)
    #ls=np.array(ls)
    sum_score=sum(ls,0)
    return(sum_score)


In [213]:
# calculate scores
#score 1
score_1_columns=['bloated_pac_qol','heavy_constipation_pac_qol','physical_discomfort_pac_qol','need_bowel_pac_qol']
score_1=calc_score(score_1_columns)
sample["component1_score_pac_qol"]=score_1

#score 2
score_2_columns=['embarassed_pac_qol','eating_less_pac_qol','careful_eat_pac_qol','decreased_app_pac_qol',
                 'worry_eat_pac_qol','bathroom_away_pac_qol','bathroom_often_pac_qol','change_routine_pac_qol']
score_2=calc_score(score_2_columns)
sample["component2_score_pac_qol"]=score_2

#score 3
score_3_columns=['irritable_pac_qol','upset_pac_qol','obsessed_pac_qol','stressed_pac_qol','less_confident_pac_qol',
                'worried_when_bowel_pac_qol','worried_not_bowel_pac_qol','more_worried_pac_qol',
                 'condition_worse_pac_qol','body_working_pac_qol']
rev_control_pac_qol=rev_score('control_pac_qol')
score3_ls=[]

for item in score_3_columns:
    arr=sample[item].to_numpy()
    int_arr = [int(i) for i in arr]
    score3_ls.append(int_arr)
        
score3_ls.append(rev_control_pac_qol)
score3_ls=np.array(score3_ls)
score3=sum(score3_ls,0)/11
sample["component3_score_pac_qol"]=score3

#score 4
score_4_reverse_columns=['satisfied_often_pac_qol','satisfied_reg_pac_qol','satisfied_time_pac_qol','satisfied_treat_pac_qol']
sum_rev_score_4=calc_reverse_scores(score_4_reverse_columns)
list_4=[]
list_4=np.array([sum_rev_score_4,calc_score(score_4_columns)])
score4=sum(list_4,0)/5
sample["component4_score_pac_qol"]=score4

#total score
list_of_scores=['']
arr1=sample['component1_score_pac_qol'].to_numpy()*4
arr2=sample['component2_score_pac_qol'].to_numpy()*8
arr3=sum(score3_ls,0)
arr4=sum(list_4,0)

score_total=sum(np.array([arr1,arr2,arr3,arr4]),0)/28
sample["total_score_pac_qol"]=score_total

In [205]:
#Replace value 
sample = sample.replace({
'patient_id': patient,
'location_pacqol': location,
'patient_assessment_of_constipation_pacqol_complete': bristol_stool_chart_complete,
'bloated_pac_qol':extremity_pacqol,
'heavy_constipation_pac_qol':extremity_pacqol,
'physical_discomfort_pac_qol':frequency_pacqol,
'need_bowel_pac_qol':frequency_pacqol,
'embarassed_pac_qol':frequency_pacqol,
'eating_less_pac_qol':frequency_pacqol,
'careful_eat_pac_qol':extremity_pacqol,
'decreased_app_pac_qol':extremity_pacqol,
'worry_eat_pac_qol':extremity_pacqol,
'bathroom_away_pac_qol':extremity_pacqol,
'bathroom_often_pac_qol':extremity_pacqol,
'change_routine_pac_qol_2':extremity_pacqol,
'irritable_pac_qol':frequency_pacqol,
'upset_pac_qol':frequency_pacqol,
'obsessed_pac_qol':frequency_pacqol,
'stressed_pac_qol':frequency_pacqol,
'less_confident_pac_qol':frequency_pacqol,
'control_pac_qol':frequency_pacqol,
'worried_when_bowel_pac_qol':extremity_pacqol,
'worried_not_bowel_pac_qol':extremity_pacqol,
'more_worried_pac_qol': extremity_pacqol,
'condition_worse_pac_qol':frequency_pacqol,
'body_working_pac_qol':frequency_pacqol,
'fewer_bowel_pac_qol':frequency_pacqol,
'satisfied_often_pac_qol':extremity_pacqol,
'satisfied_reg_pac_qol':extremity_pacqol,
'satisfied_time_pac_qol':extremity_pacqol,
'satisfied_treat_pac_qol':extremity_pacqol,
})
sample=sample.drop(['Survey Timestamp','Survey Identifier'], axis=1)


In [206]:
#switch date format to mm/dd/yyyy
#date information stored in variable year, month day
date_new=[]
sample["survey_date_pac_qol"]

for i in sample["survey_date_pac_qol"]:
    if i=='':
        date_new.append('NA')
    else:
        year=re.findall("^[0-9]{4}",i)
        month=re.findall("-[0-9]{2}-",i)[0].strip('-')
        day=re.findall("[0-9]{2}$",i)
        date_new.append(month+'/'+day[0]+'/'+year[0])
    
sample=sample.drop(columns=['survey_date_pac_qol'])
sample['survey_date_pac_qol']=date_new

#find today's date
today = datetime.datetime.today().strftime('%m-%d-%Y')

In [ ]:
#Display
pd.set_option('display.max_columns', None)
display(sample)
#Export
sample.to_csv (export_path+'patient_assessment_of_constipation_pacqol'+today+'.csv', index = False, header=True)